# Create Backend Information

In [ ]:
BUCKET = 'movie_data'
PROJECT = 'qwiklabs-gcp-ffc29df745a1ad9d'
REGION = 'European Union'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [ ]:
import tensorflow as tf
import pandas as pd
import scipy as sp


Pull data into the bucket

In [ ]:
%%bash 
#TODO need to add file location here
gsutil ls gs://$BUCKET/txtcls/eval.tsv

In [ ]:
#TODO - add right info here, loads of crap to fix
%%bash
OUTDIR=gs://${BUCKET}/txtcls/trained_finetune_native
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
REGION=us-central1
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=trainer.task \
 --package-path=${PWD}/txtclsmodel/trainer \
 --job-dir=$OUTDIR \
 --scale-tier=CUSTOM \
 --masterType: Large_model
 --runtime-version=$TFVERSION \
 -- \
 --output_dir=$OUTDIR \
# --train_data_path=gs://${BUCKET}/txtcls/train.tsv \
# --eval_data_path=gs://${BUCKET}/txtcls/eval.tsv \
# --embedding_path=gs://${BUCKET}/txtcls/glove.6B.200d.txt \
# --native \
 --num_epochs=7

In [1]:
import pandas as pd
import tensorflow as tf

from tensorflow.python.lib.io import file_io
from tensorflow.contrib.factorization import WALSMatrixFactorization

import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from google.datalab.ml import TensorBoard
print(tf.__version__)

tf.logging.set_verbosity(tf.logging.INFO)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


In [5]:
TEST_SET_RATIO = 0.9
OUTDIR = 'autoencoder_experiment'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

In [6]:
df_movies = pd.read_pickle("./data/movies.pkl")
df_rating = pd.read_pickle("./data/rating.pkl")
df_tags = pd.read_pickle("./data/tags.pkl")
df_links = pd.read_pickle("./data/links.pkl")
df_genome_scores = pd.read_pickle("./data/genome-scores.pkl")
df_genome_tags = pd.read_pickle("./data/genome-tags.pkl")

In [7]:
df_dataset = df_rating.join(df_movies, on=['movieId'], lsuffix='_rating', rsuffix='_movies',how='inner')
df_dataset = df_dataset.reset_index(drop=True)
df_dataset.columns

Index(['movieId', 'userId', 'movieId_rating', 'rating', 'timestamp',
       'movieId_movies', 'title', 'genres', 'movie_year', '(no genres listed)',
       'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [10]:
msk = np.random.rand(len(df_rating)) < 0.1
df_train = df_rating[msk]
df_test = df_rating[~msk]

print("Test length: %s , Training length: %s" % (len(df_test),len(df_train)))

Test length: 17999425 , Training length: 2000838


In [11]:
np_items = df_train.movieId.as_matrix()
np_users = df_train.userId.as_matrix()
unique_users = np.unique(np_users)
unique_items = np.unique(np_items)
n_users = unique_users.shape[0]
n_items = unique_items.shape[0]
max_item = unique_items[-1]
max_user = unique_users[-1]

if n_users != max_user or n_items != max_item:
  z = np.zeros(max_user + 1, dtype=int)
  z[unique_users] = np.arange(n_users)
  u_r = z[np_users]
  z = np.zeros(max_item + 1, dtype=int)
  z[unique_items] = np.arange(n_items)
  i_r = z[np_items]
  np_ratings = df_train.rating.as_matrix()
  ratings = np.zeros((np_ratings.shape[0], 3), dtype=object)
  ratings[:, 0] = u_r
  ratings[:, 1] = i_r
  ratings[:, 2] = np_ratings
else:
    ratings = df_train.as_matrix(['userId', 'itemId', 'rating'])
    ratings[:, 0] -= 1
    ratings[:, 1] -= 1

In [12]:
test_set_size = int(len(ratings) * TEST_SET_RATIO)
test_set_idx = np.random.choice(range(len(ratings)),
                                size=test_set_size, replace=False)
test_set_idx = sorted(test_set_idx)

ts_ratings = ratings[test_set_idx]
tr_ratings = np.delete(ratings, test_set_idx, axis=0)


u_tr, i_tr, r_tr = zip(*tr_ratings)
u_ts, i_ts, r_ts = zip(*ts_ratings)

tr_sparse = coo_matrix((r_tr, (u_tr, i_tr)), shape=(n_users, n_items))
test_sparse = coo_matrix((r_ts, (u_ts, i_ts)), shape=(n_users, n_items))

In [13]:
input_tensor = tf.SparseTensor(indices=list(zip(tr_sparse.row, tr_sparse.col)),
                                values=(tr_sparse.data).astype(np.float32),
                                dense_shape=tr_sparse.shape)

# Build AutoEncoder

In [14]:
high_weight_exponent=1.1    
    
n_inputs = 25832
h1=256
h2=16
h3=h1
n_outputs=n_inputs

my_learning_rate = 0.01

input_array = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden1=tf.layers.dense(input_array, n_inputs, activation=tf.nn.relu)
hidden2=tf.layers.dense(hidden1,h1,tf.nn.relu)
hidden3=tf.layers.dense(hidden2,h2,tf.nn.relu)
hidden4=tf.layers.dense(hidden3,h3,tf.nn.relu)
output_array=tf.layers.dense(hidden4,n_inputs)

reconstruction_loss = tf.reduce_mean((tf.losses.absolute_difference(output_array,input_array)*((input_array/5)**high_weight_exponent)))
trainStep = tf.train.AdamOptimizer(learning_rate=my_learning_rate).minimize(reconstruction_loss)
optimizer = tf.train.AdamOptimizer(my_learning_rate)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(reconstruction_loss)
init = tf.global_variables_initializer()                                      
n_iterations = 1000
codings = hidden2                                       

                                      
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={input_array: input_tensor})  # no labels (unsupervised)
    codings_val = codings.eval(feed_dict={input_array: input_tensor})
    

TypeError: float() argument must be a string or a number, not 'SparseTensor'